In [4]:
import pandas as pd
import keyring
keyring.set_password('db', 'admindb', 'Imacab@2022')
import sqlalchemy as sa
import urllib
from sqlalchemy import create_engine, event
from sqlalchemy.engine.url import URL
from flask import Flask, request, json
from flask import jsonify
    

In [ ]:
params = urllib.parse.quote_plus("DRIVER={SQL Server};"
                                 "SERVER=imacabsrv.database.windows.net;"
                                 "DATABASE=eleo_gtv;"
                                 "UID=admindb;"
                                 "PWD=" + keyring.get_password("db", "admindb") + ";")
    
engine = sa.create_engine("mssql+pyodbc:///?odbc_connect={}".format(params))
con = engine.connect()


app = Flask(__name__)


@app.route('/api/users/list', methods=['GET'])
def get_users():
    req = 'SELECT * FROM dbo.users'
    return jsonify(pd.read_sql_query(req, engine).to_dict('records'))

@app.route('/api/users/add', methods=['POST'])
def add_user():
    request_data = request.get_json()
    req = '''INSERT INTO [dbo].[users]
           ([name]
           ,[email]
           ,[mot_de_passe]
           ,[admin])
     VALUES
           ('{0}', '{1}', '{2}', '{3}')'''.format(request_data['name'], request_data['email'], 
                                                 request_data['mot_de_passe'], request_data['admin'])
    con.execute(req)
    return "200"
    

def check_connection(data):
    req = 'SELECT name, email, mot_de_passe FROM dbo.users WHERE email = \'{}\''.format(data['email'])
    users = pd.read_sql_query(req, engine).to_dict('records')
    if len(users) == 1 and users[0]['mot_de_passe'] == data['mot_de_passe']:
        return users[0]['name'], True
    return "error", False

@app.route('/api/users/connect', methods=['POST'])
def connect_req():
    request_data = request.get_json()
    res = check_connection(request_data)
    if (res[1]):
        return res[0], "200"
    return "403"
    

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [09/Nov/2022 10:38:38] "POST /api/users/connect HTTP/1.1" 200 -
127.0.0.1 - - [09/Nov/2022 10:39:02] "POST /api/users/connect HTTP/1.1" 200 -
127.0.0.1 - - [09/Nov/2022 10:39:16] "POST /api/users/connect HTTP/1.1" 200 -
